In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, isnan, min, max, mean, stddev, row_number
from pyspark.sql import functions as F
from pyspark.sql.window import Window

import numpy as np

In [ ]:
spark = SparkSession.builder.appName("BigData Hotel").getOrCreate()
spark

In [ ]:
# Đọc dữ liệu vào dataframe
df = spark.read.csv('Hotel Reservations.csv', header=True, inferSchema=True)
df.show()

+----------+------------+--------------+--------------------+-----------------+-----------------+--------------------------+------------------+---------+------------+-------------+------------+-------------------+--------------+----------------------------+------------------------------------+------------------+----------------------+--------------+
|Booking_ID|no_of_adults|no_of_children|no_of_weekend_nights|no_of_week_nights|type_of_meal_plan|required_car_parking_space|room_type_reserved|lead_time|arrival_year|arrival_month|arrival_date|market_segment_type|repeated_guest|no_of_previous_cancellations|no_of_previous_bookings_not_canceled|avg_price_per_room|no_of_special_requests|booking_status|
+----------+------------+--------------+--------------------+-----------------+-----------------+--------------------------+------------------+---------+------------+-------------+------------+-------------------+--------------+----------------------------+------------------------------------+--

In [ ]:
print("\nXem trước 5 dòng dữ liệu:")
df.show(5)


Xem trước 5 dòng dữ liệu:
+----------+------------+--------------+--------------------+-----------------+-----------------+--------------------------+------------------+---------+------------+-------------+------------+-------------------+--------------+----------------------------+------------------------------------+------------------+----------------------+--------------+
|Booking_ID|no_of_adults|no_of_children|no_of_weekend_nights|no_of_week_nights|type_of_meal_plan|required_car_parking_space|room_type_reserved|lead_time|arrival_year|arrival_month|arrival_date|market_segment_type|repeated_guest|no_of_previous_cancellations|no_of_previous_bookings_not_canceled|avg_price_per_room|no_of_special_requests|booking_status|
+----------+------------+--------------+--------------------+-----------------+-----------------+--------------------------+------------------+---------+------------+-------------+------------+-------------------+--------------+----------------------------+------------

In [ ]:
# Xem thông tin dữ liệu dưới dạng cây
df.printSchema()

root
 |-- Booking_ID: string (nullable = true)
 |-- no_of_adults: integer (nullable = true)
 |-- no_of_children: integer (nullable = true)
 |-- no_of_weekend_nights: integer (nullable = true)
 |-- no_of_week_nights: integer (nullable = true)
 |-- type_of_meal_plan: string (nullable = true)
 |-- required_car_parking_space: integer (nullable = true)
 |-- room_type_reserved: string (nullable = true)
 |-- lead_time: integer (nullable = true)
 |-- arrival_year: integer (nullable = true)
 |-- arrival_month: integer (nullable = true)
 |-- arrival_date: integer (nullable = true)
 |-- market_segment_type: string (nullable = true)
 |-- repeated_guest: integer (nullable = true)
 |-- no_of_previous_cancellations: integer (nullable = true)
 |-- no_of_previous_bookings_not_canceled: integer (nullable = true)
 |-- avg_price_per_room: double (nullable = true)
 |-- no_of_special_requests: integer (nullable = true)
 |-- booking_status: string (nullable = true)



In [ ]:
# Có nhiều cột dư thừa trong tập dữ liệu nên bỏ chúng đi trước khi tiếp tục.
df = df.drop('Booking_ID', 'arrival_year','arrival_month', 'arrival_date')
df.printSchema()

root
 |-- no_of_adults: integer (nullable = true)
 |-- no_of_children: integer (nullable = true)
 |-- no_of_weekend_nights: integer (nullable = true)
 |-- no_of_week_nights: integer (nullable = true)
 |-- type_of_meal_plan: string (nullable = true)
 |-- required_car_parking_space: integer (nullable = true)
 |-- room_type_reserved: string (nullable = true)
 |-- lead_time: integer (nullable = true)
 |-- market_segment_type: string (nullable = true)
 |-- repeated_guest: integer (nullable = true)
 |-- no_of_previous_cancellations: integer (nullable = true)
 |-- no_of_previous_bookings_not_canceled: integer (nullable = true)
 |-- avg_price_per_room: double (nullable = true)
 |-- no_of_special_requests: integer (nullable = true)
 |-- booking_status: string (nullable = true)



In [ ]:
# Thực hiện Label Encoding cho các cột nhãn được chỉ định trong DataFrame, chuyển đổi giá trị chuỗi thành chỉ số.
columns_to_check = ["room_type_reserved", "type_of_meal_plan", "market_segment_type", "booking_status"]

# Kiểm tra và hiển thị giá trị duy nhất của từng cột
for col_name in columns_to_check:
    unique_values = df.select(col_name).distinct().orderBy(col_name)
    print(f"Unique values of {col_name}:")
    unique_values.show()

# Thực hiện Label Encoding cho từng cột trong danh sách
for col_name in columns_to_check:
    # Định nghĩa cửa sổ để gán chỉ số cho các giá trị duy nhất
    window_spec = Window.orderBy(col_name)

    # Tạo DataFrame mới chứa các giá trị duy nhất và chỉ số tương ứng
    unique_df = df.select(col_name).distinct() \
                  .withColumn("index", row_number().over(window_spec) - 1)

    # Kết hợp dữ liệu gốc với bảng mã hóa và ghi đè lên cột gốc
    df = df.join(unique_df, on=col_name, how="left") \
           .drop(col_name) \
           .withColumnRenamed("index", col_name)

# Hiển thị DataFrame đã được mã hóa
df.show()

Unique values of room_type_reserved:
+------------------+
|room_type_reserved|
+------------------+
|       Room_Type 1|
|       Room_Type 2|
|       Room_Type 3|
|       Room_Type 4|
|       Room_Type 5|
|       Room_Type 6|
|       Room_Type 7|
+------------------+

Unique values of type_of_meal_plan:
+-----------------+
|type_of_meal_plan|
+-----------------+
|      Meal Plan 1|
|      Meal Plan 2|
|      Meal Plan 3|
|     Not Selected|
+-----------------+

Unique values of market_segment_type:
+-------------------+
|market_segment_type|
+-------------------+
|           Aviation|
|      Complementary|
|          Corporate|
|            Offline|
|             Online|
+-------------------+

Unique values of booking_status:
+--------------+
|booking_status|
+--------------+
|      Canceled|
|  Not_Canceled|
+--------------+

+------------+--------------+--------------------+-----------------+--------------------------+---------+--------------+----------------------------+------------

In [ ]:
# Đếm số giá trị null hoặc NaN trên từng cột
null_counts = df.select(
    [count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in df.columns]
)
null_counts.show()

+------------+--------------+--------------------+-----------------+--------------------------+---------+--------------+----------------------------+------------------------------------+------------------+----------------------+------------------+-----------------+-------------------+--------------+
|no_of_adults|no_of_children|no_of_weekend_nights|no_of_week_nights|required_car_parking_space|lead_time|repeated_guest|no_of_previous_cancellations|no_of_previous_bookings_not_canceled|avg_price_per_room|no_of_special_requests|room_type_reserved|type_of_meal_plan|market_segment_type|booking_status|
+------------+--------------+--------------------+-----------------+--------------------------+---------+--------------+----------------------------+------------------------------------+------------------+----------------------+------------------+-----------------+-------------------+--------------+
|           0|             0|                   0|                0|                         0|  

In [ ]:
# Phát hiện ngoại lệ
# Các cột định lượng
quant_columns = [
    'no_of_adults', 'no_of_children', 'no_of_weekend_nights',
    'no_of_week_nights', 'lead_time', 'no_of_previous_cancellations',
    'no_of_previous_bookings_not_canceled', 'avg_price_per_room', 'no_of_special_requests'
]

# Ngưỡng Z-score
z_score_threshold = 3

# Tính trung bình và độ lệch chuẩn
def calculate_mean_std(df, column):
    mean_val = df.agg(F.mean(column)).first()[0]
    stddev_val = df.agg(F.stddev(column)).first()[0]
    return mean_val, stddev_val

mean_std = {col: calculate_mean_std(df, col) for col in quant_columns}

# Đếm số lượng và tỉ lệ giá trị ngoại lệ trước xử lý
outliers_stats_before = {}
for col in quant_columns:
    mean, stddev = mean_std[col]
    outliers = df.filter((F.abs(F.col(col) - mean) / stddev > z_score_threshold))
    count_outliers = outliers.count()
    percentage = (count_outliers / df.count()) * 100
    outliers_stats_before[col] = (count_outliers, percentage)

print("Số lượng và tỉ lệ ngoại lệ trước khi xử lý:")
for col, (count, perc) in outliers_stats_before.items():
    print(f"{col}: Số lượng = {count}, Tỉ lệ = {perc:.2f}%")

Số lượng và tỉ lệ ngoại lệ trước khi xử lý:
no_of_adults: Số lượng = 155, Tỉ lệ = 0.43%
no_of_children: Số lượng = 1080, Tỉ lệ = 2.98%
no_of_weekend_nights: Số lượng = 184, Tỉ lệ = 0.51%
no_of_week_nights: Số lượng = 324, Tỉ lệ = 0.89%
lead_time: Số lượng = 376, Tỉ lệ = 1.04%
no_of_previous_cancellations: Số lượng = 140, Tỉ lệ = 0.39%
no_of_previous_bookings_not_canceled: Số lượng = 267, Tỉ lệ = 0.74%
avg_price_per_room: Số lượng = 357, Tỉ lệ = 0.98%
no_of_special_requests: Số lượng = 761, Tỉ lệ = 2.10%


In [ ]:
# Kiểm tra và xóa các dòng có no_of_adults và no_of_children đồng thời bằng 0
condition_adults_children = (F.col("no_of_adults") == 0) & (F.col("no_of_children") == 0)
invalid_adults_children_count = df.filter(condition_adults_children).count()
print(f"Số dòng có no_of_adults và no_of_children đồng thời bằng 0: {invalid_adults_children_count}")

if invalid_adults_children_count > 0:
    df = df.filter(~condition_adults_children)

Số dòng có no_of_adults và no_of_children đồng thời bằng 0: 0


In [ ]:
df.printSchema()

root
 |-- no_of_adults: integer (nullable = true)
 |-- no_of_children: integer (nullable = true)
 |-- no_of_weekend_nights: integer (nullable = true)
 |-- no_of_week_nights: integer (nullable = true)
 |-- required_car_parking_space: integer (nullable = true)
 |-- lead_time: integer (nullable = true)
 |-- repeated_guest: integer (nullable = true)
 |-- no_of_previous_cancellations: integer (nullable = true)
 |-- no_of_previous_bookings_not_canceled: integer (nullable = true)
 |-- avg_price_per_room: double (nullable = true)
 |-- no_of_special_requests: integer (nullable = true)
 |-- room_type_reserved: integer (nullable = true)
 |-- type_of_meal_plan: integer (nullable = true)
 |-- market_segment_type: integer (nullable = true)
 |-- booking_status: integer (nullable = true)



In [ ]:
df1 = df.select(quant_columns).describe()
df1.show()

+-------+------------------+-------------------+--------------------+------------------+-----------------+----------------------------+------------------------------------+------------------+----------------------+
|summary|      no_of_adults|     no_of_children|no_of_weekend_nights| no_of_week_nights|        lead_time|no_of_previous_cancellations|no_of_previous_bookings_not_canceled|avg_price_per_room|no_of_special_requests|
+-------+------------------+-------------------+--------------------+------------------+-----------------+----------------------------+------------------------------------+------------------+----------------------+
|  count|             36275|              36275|               36275|             36275|            36275|                       36275|                               36275|             36275|                 36275|
|   mean|1.8449620951068229|0.10527911784975878|   0.810723638869745|2.2043004824259134|85.23255685733976|        0.023349414197105445|     

In [ ]:
nor_columns = ['no_of_adults', 'no_of_children', 'no_of_weekend_nights',
    'no_of_week_nights', 'lead_time', 'no_of_previous_cancellations',
    'no_of_previous_bookings_not_canceled', 'avg_price_per_room', 'no_of_special_requests', 'room_type_reserved', 'type_of_meal_plan', 'market_segment_type']
def normalize_min_max(df, columns):
    for col in columns:
        # Tính giá trị min và max của từng cột
        min_val = df.agg(F.min(col)).first()[0]
        max_val = df.agg(F.max(col)).first()[0]

        # Kiểm tra tránh chia cho 0
        if max_val != min_val:
            # Chuẩn hóa giá trị của cột
            df = df.withColumn(
                col,
                (F.col(col) - min_val) / (max_val - min_val)
            )
        else:
            # Nếu max_val = min_val, gán tất cả giá trị của cột thành 0
            df = df.withColumn(col, F.lit(0))

    return df

# Chuẩn hóa các cột định lượng
df = normalize_min_max(df, nor_columns)

# Hiển thị một số dòng để kiểm tra kết quả
df.show()

+------------+--------------+--------------------+--------------------+--------------------------+--------------------+--------------+----------------------------+------------------------------------+-------------------+----------------------+------------------+------------------+-------------------+--------------+
|no_of_adults|no_of_children|no_of_weekend_nights|   no_of_week_nights|required_car_parking_space|           lead_time|repeated_guest|no_of_previous_cancellations|no_of_previous_bookings_not_canceled| avg_price_per_room|no_of_special_requests|room_type_reserved| type_of_meal_plan|market_segment_type|booking_status|
+------------+--------------+--------------------+--------------------+--------------------------+--------------------+--------------+----------------------------+------------------------------------+-------------------+----------------------+------------------+------------------+-------------------+--------------+
|         0.5|           0.0| 0.14285714285714285

In [ ]:
# Đếm và hiển thị giá trị riêng biệt cho từng cột
for col_name in nor_columns:
    print(f'\nGiá trị riêng biệt trong cột {col_name}:')
    # Nhóm theo giá trị và đếm
    distinct_counts = df.groupBy(col_name).count().orderBy(col_name)

    # Hiển thị kết quả
    distinct_counts.show()


Giá trị riêng biệt trong cột no_of_adults:
+------------+-----+
|no_of_adults|count|
+------------+-----+
|         0.0|  139|
|        0.25| 7695|
|         0.5|26108|
|        0.75| 2317|
|         1.0|   16|
+------------+-----+


Giá trị riêng biệt trong cột no_of_children:
+--------------+-----+
|no_of_children|count|
+--------------+-----+
|           0.0|33577|
|           0.1| 1618|
|           0.2| 1058|
|           0.3|   19|
|           0.9|    2|
|           1.0|    1|
+--------------+-----+


Giá trị riêng biệt trong cột no_of_weekend_nights:
+--------------------+-----+
|no_of_weekend_nights|count|
+--------------------+-----+
|                 0.0|16872|
| 0.14285714285714285| 9995|
|  0.2857142857142857| 9071|
| 0.42857142857142855|  153|
|  0.5714285714285714|  129|
|  0.7142857142857143|   34|
|  0.8571428571428571|   20|
|                 1.0|    1|
+--------------------+-----+


Giá trị riêng biệt trong cột no_of_week_nights:
+--------------------+-----+
|   no_of_

In [ ]:
df.toPandas().to_csv('Hotel_KNN.csv', index=0)